In [1]:
import sqlite3
import os
from datetime import date
from bs4 import BeautifulSoup
from urllib.request import urlopen
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

In [31]:
# conn = sqlite3.connect('C:/Users/ITSC/Desktop/Project/MiniProject1/noodle_street/web/db.sqlite3')
conn = sqlite3.connect("C:/Users/Jayden Jeong/OneDrive/바탕 화면/project/Mini_project_1/git/web/db.sqlite3")
cur = conn.cursor()

In [16]:
cur.execute('SELECT * FROM users_user')
tables = cur.fetchall()
print(tables)

[(1, 'pbkdf2_sha256$390000$xM4xz7jyachDrBFgRL1GY9$YEzRB4ZapW9QjSbwujWLN49vYLNK677B9JCrjO+7jzo=', '2024-02-15 01:54:54.746576', 1, 'chris', '상일', '황', 'sangil9959@gmail.com', 1, 1, '2024-02-14 15:13:36', '1995-02-16', ''), (2, 'pbkdf2_sha256$390000$p8vaWRagmVIeIWOcJccVfy$xEb6wG+YEnxlP9GXCzoEACjB6lOzDRbn4oOv1hgPCkM=', '2024-02-15 02:01:35.321710', 0, 'chris1', '상일', '황', 'sangil9959@naver.com', 0, 1, '2024-02-15 01:54:33.867212', '1995-02-16', 'users/profile/따봉티콘.jfif'), (3, 'password123', None, 0, 'john_doe', 'John', 'Doe', 'john@example.com', 0, 1, '2024-02-15', '1990-01-01', 'users/profile/default.jpg'), (4, 'password456', None, 0, 'jane_smith', 'Jane', 'Smith', 'jane@example.com', 0, 1, '2024-02-15', '1985-05-15', 'users/profile/default.jpg')]


In [4]:
cur.execute("INSERT INTO users_user ")

OperationalError: incomplete input

In [24]:
example_data = [
    ("고미태", "서울특별시 용산구 용산동2가 26-10", "용산구", "010-4876-8432", False, 5, 3, "https://wallpapercave.com/wp/wp13509956.jpg"),
    ("진주집", "서울특별시 영등포구 여의도동 36-2", "영등포구", "02-780-6108", True, 4.3, 145, "https://wallpapercave.com/wp/wp13509956.jpg")
]

In [32]:
example_data = [
    ("샘플식당1", "서울특별시 영등포구 여의도동 1-1", "영등포구", "02-1234-1234", False, 4.2, 11, "https://wallpapercave.com/wp/wp6540732.jpg")
]

In [33]:
insert_query = """
INSERT INTO single_pages_restraunt (name, address, address_gu, phone_no, parking, rating, rating_count, image)
VALUES (?, ?, ?, ?, ?, ?, ?, ?);
"""

In [34]:
for data in example_data:
    cur.execute(insert_query, data)

In [35]:
conn.commit()
cur.close()
conn.close()

In [20]:
insert_query = """
DELETE FROM single_pages_restraunt where name='고미태' or name='진주집';
"""

In [21]:
cur.execute(insert_query)

In [22]:
conn.commit()
cur.close()
conn.close()

In [187]:
hours_raw("https://www.diningcode.com/profile.php?rid=hGmRAyyZH7V3")

['02.16 (금)',
 '영업시간: 12:00 - 00:00',
 '브레이크타임: 15:30 - 17:00',
 '',
 '02.17 (토)',
 '영업시간: 12:00 - 00:00',
 '브레이크타임: 15:30 - 17:00',
 '',
 '02.18 (일)',
 '영업시간: 12:00 - 23:00',
 '브레이크타임: 15:30 - 17:00',
 '',
 '02.19 (월)',
 '영업시간: 12:00 - 23:00',
 '브레이크타임: 15:30 - 17:00',
 '',
 '02.20 (화)',
 '영업시간: 12:00 - 23:00',
 '브레이크타임: 15:30 - 17:00',
 '',
 '02.21 (수)',
 '영업시간: 12:00 - 23:00',
 '브레이크타임: 15:30 - 17:00',
 '']

In [182]:
def hours_raw(url):
    driver = webdriver.Chrome()
    driver.get(url)

    plus = driver.find_element(By.CSS_SELECTOR, "div.busi-hours > p.r > a")
    plus.click()
    driver.implicitly_wait(3)

    soup = BeautifulSoup(driver.page_source, "lxml")

    busihours = soup.select("div.busi-hours ul.list")[0].text.strip().split("\n")
    hours = [i.strip() for i in busihours if i != ""]
    
    weekdays = ["월", "화", "수", "목", "금", "토", "일"]
    
    busi_hours = []
    
    for i in range(len(hours)):
        if any(char in hours[i] for char in weekdays) and hours[i] != "휴무일":
            busi_hours.append(hours[i])

            try:
                if "확인 필요" in hours[i+1]:
                    busi_hours.append(hours[i+1])

                elif "휴무일" in hours[i+1]:
                    busi_hours.append(hours[i+1])

                elif "영업시간" in hours[i+1]:
                    busi_hours.append(hours[i+1])

            except:
                busi_hours.append("")                    
                    

            try:
                if "브레이크타임" in hours[i+2]:
                    busi_hours.append(hours[i+2])

                    
                    try:
                        if "라스트오더" in hours[i+3]:
                            busi_hours.append(hours[i+3])

                        else:
                            busi_hours.append("") 

                    except:
                        busi_hours.append("")
                    
                elif "라스트오더" in hours[i+2]:
                    busi_hours.append("")
                    busi_hours.append(hours[i+2])

                else:
                    busi_hours.append("")
                    busi_hours.append("")

            except:
                busi_hours.append("")    
                busi_hours.append("")

    return busi_hours